## Import Packages

In [2]:
import os
import glob
import numpy as np
import pandas as pd
#import network_fcon as fc
import scipy as sp
from scipy.stats import pearsonr
from scipy.stats import linregress
import seaborn as sns
import matplotlib.pyplot as plt
import re

### Set Variables and Paths

In [6]:
# Fieldstrength
fieldstrength = '7T'

# Set paths
path = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/fmri/postprocessed/" + fieldstrength + "/xcp_d"
outpath = "~/Desktop/ImageData/PMACS_remote/analysis/postprocessing/"
clinpath = "~/Desktop/ImageData/PMACS_remote/data/clinical"
cestpath = "/Users/pecsok/Desktop/ImageData/PMACS_remote/data/cest/output_measures/UNI/"

# Choose what to analyse
diag_scores = ["dx_pscat", "hstatus","sex", "age", "race","ethnic"] # ,"axis1_desc1","axis1_stat1"
diag_details = ["axis1_desc1", "axis1_desc2", "axis1_desc3","axis1_desc4","axis1_desc5", "axis1_desc6"]

# Initialize empty lists and vars
bblids = []
sesids = []

# Import group dataframes and set indices
subjlist = pd.read_csv("~/Desktop/ImageData/PMACS_remote/data/pipeline_input_all_subj.csv", sep=',') 
cestmat = pd.read_csv("cest_parcelmat" + fieldstrength + ".csv", sep=',') 
fcmat = pd.read_csv("fc_parcelmat_" + fieldstrength + ".csv", sep=',')
rehomat = pd.read_csv("reho_parcelmat" + fieldstrength + ".csv", sep=',')

# Reformat some dfs
cestmat.set_index('BBLID', inplace = True)
fcmat.set_index('BBLID', inplace = True)
rehomat.set_index('BBLID', inplace = True)
subjlist = subjlist.dropna(how='all')
subjlist.rename(columns={'fMRI Field Strength': 'fieldstrength'},inplace=True)

# Correct subject list for future! 
subjlist.loc[subjlist['BBLID'] == 88608.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 20792.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 88760.0, 'SCANID_rs'] = 'motive2'
subjlist.loc[subjlist['BBLID'] == 112126.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 91962.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 89279.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 91422.0, 'SCANID_rs'] = 'motive1'
subjlist.loc[subjlist['BBLID'] == 121407.0, 'SCANID_rs'] = '9257'
print(subjlist)  

      BBLID                  PROTOCOL_CEST DOSCAN_CEST  SCANID_CEST  lagtime  \
0     20303           843329 - LongGluCEST      6/2/22        12234      -85   
1     90217           843329 - LongGluCEST     5/19/22        12230       90   
2     88608           843329 - LongGluCEST     3/22/22        12108       55   
3     21874           843329 - LongGluCEST      3/8/22        12094       42   
4     94288           843329 - LongGluCEST      3/4/22        12092      126   
..      ...                            ...         ...          ...      ...   
119   90877  825940 - GluCEST in Psychosis     8/17/18        10907        0   
120   92155  825940 - GluCEST in Psychosis     11/9/18        11022        0   
121   93274  825940 - GluCEST in Psychosis     4/19/18        10765        0   
122  112126  825940 - GluCEST in Psychosis      2/6/19        11157        0   
123  139272  825940 - GluCEST in Psychosis     3/29/18        10739        0   

    DOSCAN_rs SCANID_rs                

In [13]:
# Identify parcels that have enough data
columns = cestmat.columns[cestmat.notnull().sum() > len(cestmat)/2]
parcels = [col.split('_')[1] for col in columns if '_' in col]

Index(['Unnamed: 0', 'Session', 'NZMean_501', 'NZcount_501', 'NZMean_502',
       'NZcount_502', 'NZMean_504', 'NZcount_504', 'NZMean_518', 'NZcount_518',
       ...
       'NZMean_961', 'NZcount_961', 'NZMean_962', 'NZcount_962', 'NZMean_972',
       'NZcount_972', 'NZMean_973', 'NZcount_973', 'NZMean_974',
       'NZcount_974'],
      dtype='object', length=220)
['501', '501', '502', '502', '504', '504', '518', '518', '521', '521', '532', '532', '540', '540', '541', '541', '542', '542', '544', '544', '545', '545', '552', '552', '555', '555', '558', '558', '560', '560', '561', '561', '562', '562', '563', '563', '564', '564', '567', '567', '574', '574', '580', '580', '581', '581', '587', '587', '590', '590', '595', '595', '596', '596', '598', '598', '606', '606', '608', '608', '613', '613', '692', '692', '707', '707', '709', '709', '712', '712', '750', '750', '751', '751', '752', '752', '754', '754', '755', '755', '757', '757', '759', '759', '761', '761', '762', '762', '763', '763', '7